## Import

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from functools import reduce

import numpy as np
import pandas as pd
import re, time
import requests
import pickle
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import os
os.getcwd()

'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\ipynb'

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

* crawling

In [4]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

* agricultural 

In [24]:
start_day = "2012/01/01"

In [25]:
cont = ['germany', 'u.s.', 'brazil', 'switzerland', 'uk', 'italy', 'japan',
       'china', 'canada', 'france', 'south-korea', 'australia']

In [26]:
urls = ['https://kr.investing.com/rates-bonds/{}-1-year-bond-yield-historical-data'.format(x) for x in cont]
urls.extend(['https://kr.investing.com/rates-bonds/{}-10-year-bond-yield-historical-data'.format(x) for x in cont])

In [27]:
names = [(x.split("/")[-1]).split("-")[0] + "_bond" for x in urls]

In [28]:
start_idx = 0

In [29]:
# inner loop
index_list = []

for idx, url in tqdm(enumerate(urls[start_idx:])):
    driver.get(url)
    time.sleep(5)

    # 날짜 입력란
    tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
    tmp_date.click()
    driver.implicitly_wait(3)

    tmp_input = driver.find_element_by_css_selector("input#startDate")
    time.sleep(0.5)

    # 기존 날짜 지우기
    for _ in range(11) :
        tmp_input.send_keys(Keys.BACKSPACE)
        tmp_input = driver.find_element_by_css_selector("input#startDate")
        time.sleep(0.5)

    tmp_input.send_keys(start_day)

    # 요청 버튼
    request_btn = driver.find_element_by_css_selector("a#applyBtn")
    request_btn.click()
    time.sleep(5)

    tables = driver.find_elements_by_css_selector("table#curr_table")

    values = tables[0].text.split("\n")[1:]

    values = [re.sub("[^0-9a-zA-Z.\\- ]","",x) for x in values]

    name = names[idx]

    index_dict = {"date":[],
                 "{}_close".format(name):[],
                 "{}_var".format(name):[]}

    for val in values:
        try :
            year, month, day, close, _, _, _, variation= val.split(" ")
        except ValueError:
            year, month, day, close, _, _, _, vol, variation= val.split(" ")

        date = "-".join([year, month, day])
        index_dict['date'].append(date)
        index_dict['{}_close'.format(name)].append(close)
        index_dict['{}_var'.format(name)].append(variation)

    index_list.append(pd.DataFrame.from_dict(index_dict, orient = "index").T)
    start_idx += 1

index_df = reduce(lambda  left,right: pd.merge(left,right,on=['date'], how='inner'), index_list)

24it [09:22, 23.43s/it]


In [32]:
index_df.tail()

,date,germany_bond_close_x,germany_bond_var_x,u.s._bond_close_x,u.s._bond_var_x,brazil_bond_close_x,brazil_bond_var_x,switzerland_bond_close_x,switzerland_bond_var_x,uk_bond_close_x,...,china_bond_close_y,china_bond_var_y,canada_bond_close_y,canada_bond_var_y,france_bond_close_y,france_bond_var_y,south_bond_close_y,south_bond_var_y,australia_bond_close_y,australia_bond_var_y
2144,2012-01-11,-0.014,-1500.00,0.1030,10.75,10.089,0.30,0.250,0.00,0.4280,...,3.450,1.47,1.940,-2.07,3.173,-3.00,3.790,0.00,3.920,0.26
2145,2012-01-10,0.001,-101.28,0.0930,-3.12,10.059,-0.49,0.250,0.00,0.4310,...,3.400,-0.58,1.981,1.33,3.271,-1.18,3.790,0.00,3.910,0.98
2146,2012-01-06,-0.112,64.71,0.0960,-4.95,10.129,-0.72,0.250,0.00,0.4210,...,3.514,1.86,1.938,-1.72,3.393,0.27,3.780,0.00,3.932,0.77
2147,2012-01-05,-0.068,21.43,0.1010,-1.94,10.202,0.13,0.250,25.00,0.4030,...,3.450,-1.43,1.972,-1.15,3.384,2.05,3.780,-0.26,3.902,-0.20
2148,2012-01-04,-0.056,229.41,0.1030,-2.83,10.189,0.69,0.200,-83.05,0.3730,...,3.500,-1.41,1.995,0.10,3.316,1.47,3.790,0.00,3.910,-1.09


In [31]:
index_df.to_csv(data_path + "bond_df.csv", index = False)